In [22]:
import easyaccess as ea
import pandas as pd
import numpy as np
import ephem

In [23]:
db = ea.connect(section = 'desoper')
cursor = db.cursor()

Connecting to DB ** desoper ** ...


In [24]:
query = "select distinct o.EXPNUM,c.CCDNUM,e.MJD_OBS,o.FILENAME,o.RA,o.DEC,o.FLUX_AUTO,o.FLUXERR_AUTO,o.FLAGS,o.IMAFLAGS_ISO,o.BAND from prod.proctag t,prod.catalog c,prod.se_object o,prod.exposure e,prod.finalcut_eval f where tag= 'Y3A1_FINALCUT' and t.pfw_attempt_id=c.pfw_attempt_id and c.filename=o.filename and o.expnum=e.expnum and f.expnum=e.expnum and f.accepted='True' and o.BAND!='Y' and o.ra between 0.00 and 0.01 and o.dec between 0.00 and 0.01 and e.exptime>80.0 order by o.EXPNUM,c.CCDNUM;"

In [101]:
result = db.query_to_pandas(query)
result.head()

,EXPNUM,CCDNUM,MJD_OBS,FILENAME,RA,DEC,FLUX_AUTO,FLUXERR_AUTO,FLAGS,IMAFLAGS_ISO,BAND
0,229683,46,56537.257498,D00229683_g_c46_r2356p01_red-fullcat.fits,0.009281,0.004538,1755.540039,258.944000,0,0,g
1,232823,36,56546.270851,D00232823_g_c36_r2357p02_red-fullcat.fits,0.009297,0.004477,2287.679932,277.850006,0,0,g
2,243108,36,56577.118535,D00243108_i_c36_r2363p01_red-fullcat.fits,0.007025,0.000913,6418.729980,771.127991,0,0,i
3,243108,36,56577.118535,D00243108_i_c36_r2363p01_red-fullcat.fits,0.008330,0.005176,3094.830078,531.112976,0,0,i
4,243108,36,56577.118535,D00243108_i_c36_r2363p01_red-fullcat.fits,0.009284,0.004464,9154.440430,655.309998,0,0,i


In [121]:
shouldbestr = str(result['EXPNUM'][3])
maybestr = ('"+result['EXPNUM'][3]"')
print type(shouldbestr)
print type(maybestr)

<type 'str'>
<type 'str'>


In [108]:
#No observations with the same RA, DEC w diff expnum

for k in result.index:
    for i in result.index:
        if result['RA'][k] == result['RA'][i] and result['DEC'][k] == result['DEC'][i] and result['EXPNUM'][k] != result['EXPNUM'][i]:
            print result['EXPNUM'][k]+" "+result['EXPNUM'][i]

In [136]:
def SEtoDB(df):
    db=ea.connect(section='desoper')
    cmd = "INSERT INTO scheibne.linker (canid, objid) VALUES ('"+df['CANID']+"', '"+str(df['OBJID'])+"')"

    cursor.execute(cmd)
    
    cmd = "INSERT INTO scheibne.Objects (date_obs, ra, dec, expnum, band, ccd, mag, objid)\
    VALUES ('"+str(df['DATE'])+"', '"+str(df['RA'])+"','"+str(df['DEC'])+"' ,'"+str(df['EXPNUM'])+"','"+str(df['BAND'])+"','"+str(df['CCD'])+"','"+str(df['MAG'])+"','"+str(df['OBJID'])+"')"
    #cursor.execute(cmd)

In [26]:
def findPoints(ra, dec):
    RA = np.degrees(ephem.hours(ra))
    DEC= np.degrees(ephem.degrees(dec))
    print RA
    print DEC
    db=ea.connect(section='desoper')
    query="select distinct o.EXPNUM,c.CCDNUM,e.MJD_OBS,e.date_obs,o.FILENAME,o.RA,o.DEC,o.FLUX_AUTO,o.FLUXERR_AUTO, \
o.FLAGS,o.IMAFLAGS_ISO,o.BAND from prod.proctag t,prod.catalog c,prod.se_object o,prod.exposure e,prod.finalcut_eval f \
where tag= 'Y3A1_FINALCUT' and t.pfw_attempt_id=c.pfw_attempt_id and c.filename=o.filename and o.expnum=e.expnum and \
f.expnum=e.expnum and f.accepted='True' and o.BAND!='Y' and o.ra between "+str(RA)+"-0.008 and "+str(RA)+"+0.008 and \
o.dec between "+str(DEC)+"-0.008 and "+str(DEC)+"+0.008 and e.exptime>80.0 order by o.EXPNUM,c.CCDNUM"
    results=db.query_to_pandas(query)
    return results

In [98]:
def findUniqueRADEC(expnum):
    i=0
    k=0
    
    for i in result.index:
        probExpnum = result['EXPNUM'][i]
        #print probExpnum
        if probExpnum == expnum:
            probRA = result['RA'][i]
            probDEC = result['DEC'][i]
                
    for k in result.index:
        falseExpnum = result['EXPNUM'][k]
        if (probRA == result['RA'][k]) and (probDEC == result['DEC'][k]) and (probExpnum != falseExpnum):
                    print 'Not unique'
                    break
                    
        else:
            print result.ix[i]
            #print probRA, probDEC
            break
           
                    
            
    

In [99]:
findUniqueRADEC(243108)

EXPNUM                                             377764
CCDNUM                                                 14
MJD_OBS                                           56976.1
FILENAME        D00377764_z_c14_r2262p01_red-fullcat.fits
RA                                               0.009283
DEC                                              0.004473
FLUX_AUTO                                         8728.08
FLUXERR_AUTO                                      1134.99
FLAGS                                                   0
IMAFLAGS_ISO                                            0
BAND                                                    z
Name: 33, dtype: object
